# vorbereitung:

In [1]:
import pandas as pd
import os
salarys = pd.read_csv("Employee_Salary_Compensation.csv", sep = ';')

In [2]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

In [3]:
X = salarys[["Year","Organization Group Code", "Department Code", "Job code"]]
y = salarys[["Salaries"]]

In [4]:
oe = preprocessing.OrdinalEncoder().fit(X[["Department Code"]])
salarys[["Department Code"]] = oe.transform(X[["Department Code"]])

In [5]:
oe = preprocessing.OrdinalEncoder().fit(X[["Job code"]])
salarys[["Job code"]] = oe.transform(X[["Job code"]])

In [6]:
X = salarys[["Year","Organization Group Code", "Department Code", "Job code"]]
y = salarys[["Salaries"]]

In [7]:
from sklearn.impute import SimpleImputer
import numpy as np
si = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
si.fit(X)
X_ = si.transform(X)
X = pd.DataFrame(X_, columns=X.columns)
len(X[X.isna().any(axis=1)])

0

In [8]:
salarys[["Salaries"]].astype(int)

,Salaries
0,30000
1,30000
2,30001
3,30001
4,30003
...,...
510232,577632
510233,584297
510234,592394
510235,596615


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
scaler = StandardScaler()
scaler = scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


# 3.1


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size = 0.2, random_state = 42
)

len(X_train), len(X_test)

(408189, 102048)

# 3.2

In [11]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMRegressor
from xgboost.sklearn import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor



algorithms = {
    #"LinearRegression" : LinearRegression(),
    #"LGBMRegressor" : LGBMRegressor(),
    #"SGDRegressor" : SGDRegressor(max_iter=1000),
    #"GradientBoostingRegressor" : GradientBoostingRegressor(),
    #"DecisionTreeRegressor" : DecisionTreeRegressor(max_depth=30),
    "RandomForestRegressor" : RandomForestRegressor(max_depth=45, random_state=0)
}

for name, algorithm in algorithms.items():
    algorithm.fit(X_train, y_train.values.ravel())
    score = algorithm.score(X_test, y_test)
    print(name, round(score,2))

C:\Users\untid\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


LinearRegression 0.04
LGBMRegressor 0.57
SGDRegressor -4.084227739852826e+18
GradientBoostingRegressor 0.43
DecisionTreeRegressor 0.7
RandomForestRegressor 0.71


Ich habe mich für den Random Forest regressor entschieden da er den höchsten score erzielt hat.

# 3.3

In [23]:
import joblib
joblib.dump(algorithms["RandomForestRegressor"], 'salaries_random_forest.joblib')
best_model = joblib.load('salaries_random_forest.joblib')
best_model

RandomForestRegressor(max_depth=45, random_state=0)

In [24]:
test1 = pd.DataFrame([[2013.0,2.0,41.0,1227.0]], columns=X_train.columns)
test1.head()
test2 = pd.DataFrame([[2020.0,5.0,39.0,663.0]], columns=X_train.columns)
test2.head()

,Year,Organization Group Code,Department Code,Job code
0,2020.0,5.0,39.0,663.0


In [25]:
predictions = best_model.predict(pd.concat([test1,test2]))
for person, prediction in zip(["test1","test2"], predictions):
    print(prediction)

61520.57254997877
35199.43110472036


In [26]:
X.head(1000)
y.head(1000)


,Salaries
0,30000.83
1,30000.99
2,30001.03
3,30001.28
4,30003.97
...,...
995,30367.60
996,30368.53
997,30368.63
998,30369.20


Er ist nicht gerade genau, aber ich glaube nicht, dass das an den Algorithmus liegt, sondern eher an der Datenbank da es sein kann das diese nicht ganz ein Jahr gearbeitet haben und darum das Resultat verzogen wird. ich bin darauf gekommen da das zweite Resultat viel akkurater ist das ist wahrscheinlich darum das der Erste ein falscher wert war und der Zweite ein richtiger.